In [2]:
import numpy as np
import scipy as sp

import PyPO

In normalized field units of $\sqrt{\mathrm{Watts}}$ for all fields, the field at a point $\mathbf{r}$ due to the induced currents on a scatterer are given by:

$$ \mathbf{E}(\mathbf{r}) = - k Z_m \int_{S} \Big( 1 + \frac{1}{k^2}\mathbf{\nabla}\mathbf{\nabla} \Big) G(\mathbf{r}, \mathbf{r_q}) \cdot \mathbf{J}(\mathbf{r_s}) d s_q + \int_{S} \mathbf{M}(\mathbf{r_q}) \times \mathbf{\nabla} G(\mathbf{r}, \mathbf{r_q}) d s_q $$

$$ \mathbf{H}(\mathbf{r}) = - k Z_m \int_{S} \Big( 1 + \frac{1}{k^2}\mathbf{\nabla}\mathbf{\nabla} \Big) G(\mathbf{r}, \mathbf{r_q}) \cdot \mathbf{M}(\mathbf{r_s}) d s_q + \int_{S} \mathbf{J}(\mathbf{r_q}) \times \mathbf{\nabla} G(\mathbf{r}, \mathbf{r_q}) d s_q $$


For propagation in freespace (or a lossless dielectric), these equations reduce to:

$$ \mathbf{E}(\mathbf{r}) =  \int_{S} \frac{k^2}{4\pi} e^{-ikR} \Bigg[ Z_m \Bigg[ \mathbf{J} (\mathbf{r_s}) \Bigg(\frac{-i}{k R} - \frac{1}{k^2 R^2} + \frac{i}{k^3 R^3} \Bigg)  + \Big(\mathbf{J} (\mathbf{r_s})\cdot\mathbf{\hat{R}}\Big) \mathbf{\hat{R}} \Bigg( \frac{i}{kR} + \frac{3}{k^2 R^2} - \frac{3 i}{k^3 R^3} \Bigg) \Bigg] $$
$$ - \mathbf{M}(\mathbf{r}) \times \mathbf{\hat{R}} \Bigg( \frac{1}{k^2 R^2} + \frac{i}{kR}\Bigg) \Bigg] ds $$

$$ \mathbf{H}(\mathbf{r}) = \int_{S} \frac{k^2}{4\pi} e^{-ikR} \Bigg[ \frac{1}{Z_m} \Bigg[ \mathbf{M} (\mathbf{r_s}) \Bigg(\frac{-i}{k R} - \frac{1}{k^2 R^2} + \frac{i}{k^3 R^3} \Bigg)  + \Big(\mathbf{M} (\mathbf{r_s})\cdot\mathbf{\hat{R}}\Big) \mathbf{\hat{R}} \Bigg( \frac{i}{kR} + \frac{3}{k^2 R^2} - \frac{3 i}{k^3 R^3} \Bigg) \Bigg] $$
$$ + \mathbf{J}(\mathbf{r}) \times \mathbf{\hat{R}} \Bigg( \frac{1}{k^2 R^2} + \frac{i}{kR}\Bigg) \Bigg] ds $$

where the integral is carried out over the surface of the source scatterer $S$ and:

* $ds$ is the area element of the source scatterer.
* $\mathbf{r}$ is the point at which the field is to be calculated.
* $\mathbf{r}_s$ is the point on the source scatterer.
* $\mathbf{R} = \mathbf{r} - \mathbf{r}_s$ is the vector distance from the source element to the target point.
* $R = |\mathbf{R}|$.
* $\mathbf{\hat{R}} = \mathbf{R} / R$ is the unit vector along $\mathbf{R}$.
* $k$ is the wavenumber in the medium.
* $Z_m = \sqrt{\mu / \epsilon}$ is the impedance of the medium.
* $\mathbf{J}_s$ is the electric current at the source scatterer.
* $\mathbf{M}_s$ is the magnetic current at the source scatterer.


The following code implements the integrand of each of the above surface integrals.

Where the normal of the source current element points away from the target point, the integrand returns zero. This implements the shading of the target by the source scatterer.

For scatterers with surface and bulk properties that allow for transmission through the scatterer, we would instead return the field contribution from the current element on the back side of the scatterer.

In [3]:
def E_field_integrand(target_point, source_point, ns, As, js, ms, k, Z0):
    """Calculate the integrand for the surface integral over the source used in calculating
    the E field at a target_point.
    
    @param target_point np.ndarray(<float>)   The point at which to calculate the field
    @param source_point np.ndarray(<float>)   The center of the current element that contributes to the integral
    @param ns           np.ndarray(<float>)   The normal to the source surface at the center of the current element
    @param As           float                 The area of the source current element
    @param js           np.ndarray(<complex>) The electric current at the center of the current element
    @param ms           np.ndarray(<complex>) The magnetic current at the center of the current element
    @param k            float                 The wavenumber for the calculation in m^-1
    @param Z0           float                 The impedance of the propagation medium (377 Ohm for freespace)"""
    R_vec = source_point - target_point
    R = np.sqrt(np.vecdot(R_vec, R_vec))
    R_hat = R_vec/R
    
    # Check to see if the source current element points towards the target
    # This handles shading
    norm_dot_R = np.vecdot(ns, R_hat)
    if norm_dot_R > 0:
        return np.zeros_like(js)
    
    kR = k * R
    kR_inv = 1/kR  # This is a handy optimization for both the look of the code and performance
    
    kR_inv_sum1 = -kR_inv**2 + 1j*kR_inv*(kR_inv**2 - 1)
    kR_inv_sum2 = 3*kR_inv**2 + 1j*kR_inv*(1 - 3*kR_inv**2)
    kR_inv_sum3 = kR_inv**2 + 1j*kR_inv
    
    js_dot_R = np.vecdot(R_hat, js)
    
    js_dot_R_R = R_hat*js_dot_R
    
    ms_cross_R = np.linalg.cross(R_hat, ms)
    
    green = k**2 / 4*np.pi * np.exp(-1j*kR) * As
    
    d_e_field = (Z0 * (js*kR_inv_sum1 + js_dot_R_R*kR_inv_sum2) - ms_cross_R*kR_inv_sum3) * green
    
    return d_e_field

    
def H_field_integrand(target_point, source_point, ns, As, js, ms, k, Z0):
    """Calculate the integrand for the surface integral over the source used in calculating
    the E field at a target_point.
    
    @param target_point np.ndarray(<float>)   The point at which to calculate the field
    @param source_point np.ndarray(<float>)   The center of the current element that contributes to the integral
    @param ns           np.ndarray(<float>)   The normal to the source surface at the center of the current element
    @param As           float                 The area of the source current element
    @param js           np.ndarray(<complex>) The electric current at the center of the current element
    @param ms           np.ndarray(<complex>) The magnetic current at the center of the current element
    @param k            float                 The wavenumber for the calculation in m^-1
    @param Z0           float                 The impedance of the propagation medium (377 Ohm for freespace)"""
    R_vec = source_point - target_point
    R = np.sqrt(np.vecdot(R_vec, R_vec))
    R_hat = R_vec/R
    
    # Check to see if the source current element points towards the target
    # This handles shading
    norm_dot_R = np.vecdot(ns, R_hat)
    if norm_dot_R > 0:
        return np.zeros_like(ms)
    
    kR = k * R
    kR_inv = 1/kR
    
    kR_inv_sum1 = -kR_inv**2 + 1j*kR_inv*(kR_inv**2 - 1)
    kR_inv_sum2 = 3*kR_inv**2 + 1j*kR_inv*(1 - 3*kR_inv**2)
    kR_inv_sum3 = kR_inv**2 + 1j*kR_inv
    
    ms_dot_R = np.vecdot(ms, R_hat)
    
    ms_dot_R_R = R_hat*ms_dot_R
    
    js_cross_R = np.linalg.cross(R_hat, js)
    
    green = k**2 / 4*np.pi * np.exp(-1j*kR) * As
    
    d_h_field = (1/Z0 * (ms*kR_inv_sum1 + ms_dot_R_R*kR_inv_sum2) + js_cross_R*kR_inv_sum3) * green
    
    return d_h_field

## Evaluating the Surface Integral

Evaluation of the surface integral is the main computational challenge in performaing PO calculations.

For moderately large surfaces and non-paraxial angles between the source surface normal and the target point, the complex phase of the integrands oscilates rapidly due to the Green's function's $e^{-i k R}$ term.

The integral is performed using Gaussian quadrature in the $x$ and $y$ directions for rectangular grids and in the $\rho$ direction for polar grids.  For the $\phi$ direction of polar grids, where the two end points of the integration have the same value, the trapezoidal method is sufficiently accurate.

In the polar case, we integrate over the $\phi$ direction before the $\rho$ direction. This allows for the future use of lower numbers of PO points near the center of the scatterer.